In [16]:
import sqlite3

conn = sqlite3.connect('exportaciones.db')

cursor = conn.cursor()



In [17]:
for row in conn.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    table = row[0]
    print(f"Schema for table '{table}':")
    schema = conn.execute(f"PRAGMA table_info('{table}')").fetchall()
    for col in schema:
        print(col)
    print()

Schema for table 'Dim_Transporte':
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'tipo', 'TEXT', 0, None, 0)
(2, 'pais_bandera', 'TEXT', 0, None, 0)

Schema for table 'sqlite_sequence':
(0, 'name', '', 0, None, 0)
(1, 'seq', '', 0, None, 0)

Schema for table 'Dim_Modalidad':
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'id_modalidad', 'TEXT', 0, None, 0)
(2, 'descripcion', 'TEXT', 0, None, 0)

Schema for table 'Dim_Destino':
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'nombre_pais', 'TEXT', 0, None, 0)
(2, 'nombre_ciudad', 'TEXT', 0, None, 0)

Schema for table 'Dim_Fecha':
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'año', 'INTEGER', 0, None, 0)
(2, 'mes', 'INTEGER', 0, None, 0)
(3, 'dia', 'INTEGER', 0, None, 0)
(4, 'trimestre', 'INTEGER', 0, None, 0)
(5, 'semestre', 'INTEGER', 0, None, 0)

Schema for table 'Dim_Transaccion':
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'moneda', 'TEXT', 0, None, 0)
(2, 'fob_usd', 'REAL', 0, None, 0)

Schema for table 'Dim_Producto':
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'unidad', 'TEXT', 0,

¿Cuáles son las empresas que más exportaron en el último mes?

In [14]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("example.db")

last_year, last_month = conn.execute("""
    SELECT año, mes
    FROM Dim_Fecha
    ORDER BY año DESC, mes DESC
    LIMIT 1
""").fetchone()

query = """
SELECT emp.nit,
       emp.razon_social,
       SUM(tra.fob_usd) AS total_fob_usd
FROM Hecho_Exportacion AS he
JOIN Dim_Empresa     AS emp ON he.empresa     = emp.id
JOIN Dim_Transaccion AS tra ON he.transaccion = tra.id
JOIN Dim_Fecha       AS fec ON he.fecha       = fec.id
WHERE fec.año = ? AND fec.mes = ?
GROUP BY emp.id, emp.nit, emp.razon_social
ORDER BY total_fob_usd DESC
"""

df_empresas_ultimo_mes = pd.read_sql_query(query, conn, params=(last_year, last_month))
df_empresas_ultimo_mes


OperationalError: no such table: Dim_Fecha

¿Cuál es el precio total (valor FOB) de las mercancías exportadas mes a mes?

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("example.db")

query = """
SELECT fec.año,
       fec.mes,
       SUM(tra.fob_usd) AS total_fob_usd
FROM Hecho_Exportacion AS he
JOIN Dim_Fecha       AS fec ON he.fecha       = fec.id
JOIN Dim_Transaccion AS tra ON he.transaccion = tra.id
GROUP BY fec.año, fec.mes
ORDER BY fec.año, fec.mes
"""

df_fob_mensual = pd.read_sql_query(query, conn)
df_fob_mensual


¿Cuáles son los destinos donde más se exporta en los últimos 6 meses?

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("example.db")

query = """
WITH ultimos_meses AS (
    SELECT DISTINCT año,
           mes,
           (año * 100 + mes) AS ym
    FROM Dim_Fecha
    ORDER BY año DESC, mes DESC
    LIMIT 6
),
min_ym AS (
    SELECT MIN(ym) AS min_ym FROM ultimos_meses
)
SELECT des.nombre_pais,
       des.nombre_ciudad,
       SUM(tra.fob_usd) AS total_fob_usd
FROM Hecho_Exportacion AS he
JOIN Dim_Destino     AS des ON he.destino     = des.id
JOIN Dim_Transaccion AS tra ON he.transaccion = tra.id
JOIN Dim_Fecha       AS fec ON he.fecha       = fec.id
JOIN min_ym          AS m   ON (fec.año * 100 + fec.mes) >= m.min_ym
GROUP BY des.id, des.nombre_pais, des.nombre_ciudad
ORDER BY total_fob_usd DESC
"""

df_destinos_6m = pd.read_sql_query(query, conn)
df_destinos_6m
